## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

lines = list(open('./news_train.txt', 'r', encoding='utf-8'))

In [1]:
!pip install pymorphy2


import random
from gensim import models
from collections import Counter
import re
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import  LogisticRegression
from sklearn import model_selection
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
lines = list(open('./news_train.txt', 'r', encoding='utf-8'))
random.shuffle(lines)

In [2]:
Counter([line.split('\t')[0] for line in lines[:15000]])

Counter({'business': 554,
         'culture': 2053,
         'economics': 2080,
         'forces': 1225,
         'life': 2033,
         'media': 2111,
         'science': 2156,
         'sport': 2215,
         'style': 284,
         'travel': 289})

In [3]:
training_topics = []
training_text = []

for line in lines:
    lines_split = line.split('\t')
    training_topics.append(lines_split[0])
    training_titles = lines_split[1] 
    training_sentences = lines_split[2]
    training_text.append(training_titles + ' ' + training_sentences)

In [4]:
morph = pymorphy2.MorphAnalyzer()

In [5]:
words = []
for item in training_text:
    words.append(re.sub('[^а-я ]', '', item.replace('.', ' ').lower()).split())

Посмотрим, что мы имеем:

In [6]:
training_text[0]

'Ростовский институт ракетных войск пройдет реорганизацию На базе Ростовского военного института ракетных войск (РАУ), закрытого в 2011 году в рамках реформы вооруженных сил, может быть создан новый военный вуз. С таким заявлением, как передает РИА Новости, выступил заместитель губернатора Ростовской области Вадим Артемов.По словам Артемова, губернатор области Василий Голубев обращался к министру обороны Анатолию Сердюкову с просьбой передать часть земель РАУ Южному федеральному университету. В Минобороны губернатору отказали, пояснив, что территория вуза в дальнейшем будет использоваться по назначению.Кроме того, как рассказал агентству Артемов, Василий Голубев обращался к Сердюкову с просьбой передать часть территории также закрывшегося Новочеркасского высшего военного командного училища связи под кадетский корпус. Пошел ли министр на встречу ростовскому губернатору в этом вопросе, не уточняется.В рамках военной реформы 2008 года Минобороны планировало на базе 65 военных вузов 10 нау

In [7]:
words[0]

['ростовский',
 'институт',
 'ракетных',
 'войск',
 'пройдет',
 'реорганизацию',
 'на',
 'базе',
 'ростовского',
 'военного',
 'института',
 'ракетных',
 'войск',
 'рау',
 'закрытого',
 'в',
 'году',
 'в',
 'рамках',
 'реформы',
 'вооруженных',
 'сил',
 'может',
 'быть',
 'создан',
 'новый',
 'военный',
 'вуз',
 'с',
 'таким',
 'заявлением',
 'как',
 'передает',
 'риа',
 'новости',
 'выступил',
 'заместитель',
 'губернатора',
 'ростовской',
 'области',
 'вадим',
 'артемов',
 'по',
 'словам',
 'артемова',
 'губернатор',
 'области',
 'василий',
 'голубев',
 'обращался',
 'к',
 'министру',
 'обороны',
 'анатолию',
 'сердюкову',
 'с',
 'просьбой',
 'передать',
 'часть',
 'земель',
 'рау',
 'южному',
 'федеральному',
 'университету',
 'в',
 'минобороны',
 'губернатору',
 'отказали',
 'пояснив',
 'что',
 'территория',
 'вуза',
 'в',
 'дальнейшем',
 'будет',
 'использоваться',
 'по',
 'назначению',
 'кроме',
 'того',
 'как',
 'рассказал',
 'агентству',
 'артемов',
 'василий',
 'голубев',
 'об

In [8]:
for i, line in enumerate(words):
    for j, word in enumerate(line):
        p = morph.parse(word)[0]
        words[i][j] = p.normal_form

In [9]:
words[0]

['ростовский',
 'институт',
 'ракетный',
 'войско',
 'пройти',
 'реорганизация',
 'на',
 'база',
 'ростовский',
 'военный',
 'институт',
 'ракетный',
 'войско',
 'рау',
 'закрытый',
 'в',
 'год',
 'в',
 'рамка',
 'реформа',
 'вооружённый',
 'сила',
 'мочь',
 'быть',
 'создать',
 'новый',
 'военный',
 'вуз',
 'с',
 'такой',
 'заявление',
 'как',
 'передавать',
 'риа',
 'новость',
 'выступить',
 'заместитель',
 'губернатор',
 'ростовский',
 'область',
 'вадим',
 'артём',
 'по',
 'слово',
 'артёмов',
 'губернатор',
 'область',
 'василий',
 'голубеть',
 'обращаться',
 'к',
 'министр',
 'оборона',
 'анатолий',
 'сердюков',
 'с',
 'просьба',
 'передать',
 'часть',
 'земля',
 'рау',
 'южный',
 'федеральный',
 'университет',
 'в',
 'минобороны',
 'губернатор',
 'отказать',
 'пояснить',
 'что',
 'территория',
 'вуз',
 'в',
 'дальнейший',
 'быть',
 'использоваться',
 'по',
 'назначение',
 'кроме',
 'тот',
 'как',
 'рассказать',
 'агентство',
 'артём',
 'василий',
 'голубеть',
 'обращаться',
 'к'

In [10]:
model = models.Word2Vec(min_count = 1, alpha = 0.01, min_alpha = 0.0001, workers = 3)
model.build_vocab(words)
model.train(words, total_examples = model.corpus_count, epochs = 50)
model.wv.most_similar('спорт')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('мутко', 0.5563048124313354),
 ('футбол', 0.5459227561950684),
 ('рспорт', 0.5343116521835327),
 ('самбо', 0.5237361192703247),
 ('баскетбол', 0.519648551940918),
 ('культура', 0.5051917433738708),
 ('пкр', 0.4898017644882202),
 ('спортивный', 0.4810803532600403),
 ('волейбол', 0.4804432988166809),
 ('дзюдо', 0.4772310256958008)]

In [11]:
vector = TfidfVectorizer(preprocessor=' '.join) 
X_train = vector.fit_transform(words) 

In [12]:
classifier = LogisticRegression(max_iter = 10000)
classifier.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [13]:
parameters_grid = {'C': [0.0001, 0.001, 0.01, 0.005, 0.1, 1]}

In [14]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid)

In [15]:
d = dict(zip(list(set(training_topics)), range(len(training_topics))))
y_train = [d[key] for key in training_topics]
y_train[:5]

[8, 9, 1, 9, 6]

In [16]:
%%time
grid_cv.fit(X_train, y_train)

CPU times: user 4min 16s, sys: 2min 35s, total: 6min 52s
Wall time: 3min 33s


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=10000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.005, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [17]:
print(grid_cv.best_params_)
grid_cv.best_estimator_

{'C': 1}


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
lines = list(open('./news_test.txt', 'r', encoding='utf-8'))

In [19]:
testing_topics = []
testing_text = []

for line in lines:
    lines_split = line.split('\t')
    testing_topics.append(lines_split[0])
    testing_titles = lines_split[1]
    testing_sentences = lines_split[2]
    testing_text.append(testing_titles + testing_sentences)
words_test = []
for item in testing_text:
    words_test.append(re.sub('[^а-я ]', '', item.replace('.', ' ').lower()).split())
for i, line in enumerate(words_test):
    for j, word in enumerate(line):
        p = morph.parse(word)[0]
        words_test[i][j] = p.normal_form

X_test = vector.transform(words_test) 

y_test = [d[key] for key in testing_topics]
y_test[:5]

[3, 4, 4, 7, 3]

In [20]:
y_pred = grid_cv.best_estimator_.predict(X_test)
accuracy_score(y_test, y_pred)

0.8746666666666667

In [21]:
classifier = MultinomialNB() 
classifier.get_params().keys()  

dict_keys(['alpha', 'class_prior', 'fit_prior'])

In [22]:
parameters_grid = {'alpha': [0, 0.01, 0.1, 1]}

In [23]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid)

In [24]:
%%time
grid_cv.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


CPU times: user 1.16 s, sys: 32.2 ms, total: 1.2 s
Wall time: 1.2 s


GridSearchCV(cv=None, error_score=nan,
             estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0, 0.01, 0.1, 1]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=0)

In [25]:
print(grid_cv.best_params_)
grid_cv.best_estimator_

{'alpha': 0.01}


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [26]:
y_pred = grid_cv.best_estimator_.predict(X_test)
accuracy_score(y_test, y_pred)

0.8516666666666667